A catalog named telecom_catalog_assign
A schema landing_zone
A volume landing_vol
Using dbutils.fs.mkdirs, create folders:
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/ /Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/ /Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):
a. Volume vs DBFS/FileStore
## # b. Why production teams prefer Volumes for regulated data

In [0]:
spark.sql("CREATE CATALOG IF NOT EXISTS telecom_catalog_assign")
spark.sql("CREATE SCHEMA IF NOT EXISTS  telecom_catalog_assign.landing_zone")
spark.sql(
    "CREATE VOLUME if not exists telecom_catalog_assign.landing_zone.landing_vol"
)

# Now creat
# Then create the directory
#dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")



In [0]:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

# Define the DBFS path for the new file
dbfs_path = "dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customers.csv" # A common location for user uploads

# Use dbutils.fs.put() to write the string content to DBFS
# The 'True' argument means 'overwrite' (optional, set to True to replace existing file)
dbutils.fs.put(dbfs_path, customer_csv.strip(), True)

print(f"Data successfully written to {dbfs_path}")

In [0]:
usage_tsv ='''
customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''
dbfs_path = "dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv"
dbutils.fs.put(dbfs_path, usage_tsv.strip(), True)

print(f"Data successfully written to {dbfs_path}")

tower_logs_region1 = '''
event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
'''
dbfs_path = "dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv"
dbutils.fs.put(dbfs_path, tower_logs_region1.strip(), True)

print(f"Data successfully written to {dbfs_path}")



In [0]:
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer")
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage")
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower")



In [0]:
tower_logs_region2 = '''
event_id|customer_id|tower_id|signal_strength|timestamp
5004|104|TWR05|-75|2025-01-10 11:01:12
'''
dbfs_path = "dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region2.csv"
dbutils.fs.put(dbfs_path, tower_logs_region2.strip(), True)

print(f"Data successfully written to {dbfs_path}")

Read all tower logs using: Path glob filter (example: *.csv) Multiple paths input Recursive lookup

Demonstrate these 3 reads separately: Using pathGlobFilter Using list of paths in spark.read.csv([path1, path2]) Using .option("recursiveFileLookup","true")

Compare the outputs and understand when each should be used.

In [0]:

df_multiple_sources=spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv","dbfs:////Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region2.csv"],inferSchema=True,header=True,sep='|',pathGlobFilter="*.csv",recursiveFileLookup=True)
#.toDF("cid","fn","ln","a","p")
print(df_multiple_sources.count())
df_multiple_sources.show(2)

In [0]:
'''Demonstrate these 3 reads separately: Using pathGlobFilter Using list of paths in spark.read.csv([path1, path2]) Using .option("recursiveFileLookup","true")'''
df_multiple_sources=spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv","dbfs:////Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region2.csv"],inferSchema=True,header=True,sep='|',recursiveFileLookup=True)
print(df_multiple_sources.count())
df_multiple_sources.show(2)

In [0]:

csv_df1=spark.read.csv(path="dbfs:////Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region*.csv",inferSchema=True,header=True,sep='|')
print(csv_df1.count())
csv_df1.show()

In [0]:
csv_df1=spark.read.csv(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customers.csv",inferSchema=False,header=False,sep=',').toDF("id","name","age","loc","plan")
df2=csv_df1.where("age!='abc' ")
csv_df1.printSchema()
print(csv_df1.count())
csv_df1.show()
df2.show()

In [0]:
csv_df1=spark.read.csv(path="dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv",inferSchema=True,header=True,sep='\t')
print(csv_df1.count())
csv_df1.show()

In [0]:
df_recursive = spark.read.format("csv") .option("header", "true") .option("delimiter", "|") .option("recursiveFileLookup", "true") .load("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")

print("--- DataFrame using recursiveFileLookup='true' ---")
df_recursive.show()

In [0]:
df_glob = spark.read.format("csv") .option("header", "true")  .option("delimiter", "|") .option("pathGlobFilter", "*region*.csv") .load("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")

print("--- DataFrame using pathGlobFilter ('region*.csv') ---")
df_glob.show()

Apply column names using string using toDF function for customer data
Apply column names and datatype using the schema function for usage data
Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data

In [0]:

#customer_id|voice_mins|data_mb|sms_count
str_struct="customer_id integer,voice_mins integer,data_mb integer,sms_count integer"
csv_df1=spark.read.schema(str_struct).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv",header=True,sep='\t')
print(csv_df1.printSchema())
csv_df1.show()


#2. Important part - Using structure type to define custom complex schema.
#4000001,Kristina,Chung,55,Pilot
#pattern - 
#import the types library based classes..
# define_structure=StructType([StructField("colname",DataType(),True),StructField("colname",DataType(),True)...])



In [0]:
#event_id|customer_id|tower_id|signal_strength|          timestamp
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
custom_schema=StructType([StructField("event_id",IntegerType(),False),StructField("customer_id",IntegerType(),False),StructField("tower_id",StringType(),False),StructField("signal_strength",IntegerType(),False),StructField("timestamp",StringType(),False)])
csv_df1=spark.read.schema(custom_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/",pathGlobFilter="*.csv",header='True',sep='|')
print(csv_df1.printSchema())
csv_df1.show()